In [1]:
if __name__=='__main__':
    import notebookimport

In [2]:
if __name__=='__main__':
    sr = __import__("Charts - Stage Results")
    dc = __import__("Charts - Split Sector Delta")

importing Jupyter notebook from Charts - Stage Results.ipynb
importing Jupyter notebook from Charts - Split Sector Delta.ipynb


In [3]:
import os
import sqlite3
import pandas as pd

#dbname='wrc18.db'
dbname='mexico18.db'
conn = sqlite3.connect(dbname)

In [4]:
rally='Mexico'

In [5]:
'''year=2018
rc='RC2'
ss='SS4'


typ='stage_times_stage' #stage_times_stage stage_times_overall
typ='stage_times_overall'''

"year=2018\nrc='RC2'\nss='SS4'\n\n\ntyp='stage_times_stage' #stage_times_stage stage_times_overall\ntyp='stage_times_overall"

In [6]:
stageresults = sr.dbGetRallyStages(conn, rally)
stagerank = sr.getEnrichedStageRank(conn,rally)

In [7]:
sections = stageresults.groupby('section')

#Generate list from group: stageresults.groupby('section')['code'].apply(list)

#Generate group then iterate through groups
for key in sections.groups.keys():
    print(key, sections.get_group(key)['code'].tolist())


Section 1 ['SS1']
Section 2 ['SS2', 'SS3', 'SS4', 'SS5']
Section 3 ['SS6', 'SS7', 'SS8', 'SS9', 'SS10']
Section 4 ['SS11', 'SS13', 'SS12']
Section 5 ['SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19']
Section 6 ['SS20', 'SS21']
Section 7 ['SS22']


In [8]:
basepath = 'report'
imgdir = 'images'
imgdirfull = '{}/{}'.format(basepath, imgdir)


In [9]:
sectionREADME_base = '''\n### {section} Report
'''

In [17]:
%%capture
if not os.path.exists(imgdirfull):
    os.makedirs(imgdirfull)
    
for key in sections.groups.keys():
    #path = '{}/{}'.format(basepath,key)
    #if not os.path.exists(path):
    #    os.makedirs(path)
    sectionStages = sections.get_group(key)['code'].tolist()
    stagerank = sr.stageOverallEnrichers(stagerank)
    ssdemo = stagerank[stagerank['code'].isin(sectionStages)].copy()
    ssdemo = sr.stageLegEnrichers(ssdemo)
    fn='{}/spchart_{}.png'.format(imgdirfull, key)
    
    fig, ax = sr.plotStageProgressionChart( ssdemo, linecolor='lightgrey' , deltalabels=True, 
                                        progress=True, stageAnnotate=False, filename=fn);
    
    if fig is None: continue
    fn='{}/spchart_{}.png'.format(imgdir, key)
    
    sectionREADME = sectionREADME_base.format(section=key)
    sectionREADME = '''{s}{key}\n\n![]({img})'''.format(s=sectionREADME, key=key,img=fn)

    with open('{bp}/{key}_report.md'.format(bp=basepath,key=key), 'a') as out_file:
        out_file.write(sectionREADME)


In [18]:
%%capture
fn='{}/spchart_full.png'.format(imgdirfull)
fig, ax = sr.plotStageProgressionChart( stagerank, linecolor='lightgrey' , deltalabels=True, 
                                        progress=True, stageAnnotate=False, filename=fn);

README='''\n![]({})'''.format('{}/spchart_full.png'.format(imgdir))
with open('{}/README.md'.format(basepath), 'a') as out_file:
    out_file.write(README)

In [14]:
!ls report

README.md           Section 2_report.md Section 4_report.md Section 6_report.md
SUMMARY.md          Section 3_report.md Section 5_report.md images
